In [1]:
from vllm import LLM
import torch

In [2]:
loaded_directions = torch.load('directions.pth')



In [3]:
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1")

INFO 06-19 10:56:17 llm_engine.py:163] Initializing an LLM engine (v0.5.0.post1) with config: model='mistralai/Mistral-7B-Instruct-v0.1', speculative_config=None, tokenizer='mistralai/Mistral-7B-Instruct-v0.1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=mistralai/Mistral-7B-Instruct-v0.1)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-19 10:56:18 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-19 10:56:18 selector.py:51] Using XFormers backend.
INFO 06-19 10:56:21 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 06-19 10:56:21 selector.py:51] Using XFormers backend.
INFO 06-19 10:56:21 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 06-19 10:56:25 model_runner.py:167] Loading model weights took 13.4966 GB
INFO 06-19 10:56:29 gpu_executor.py:85] # GPU blocks: 11432, # CPU blocks: 2048
INFO 06-19 10:56:32 model_runner.py:915] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-19 10:56:32 model_runner.py:919] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. Y

In [4]:
from vllm.control_vectors.request import ControlVectorRequest
from vllm.control_vectors.control import ControlVector


cv = ControlVector(model_type="mistral", directions=loaded_directions)



In [5]:
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature=0,
    max_tokens=256,
    stop=["[/assistant]"]
)

In [6]:
prompts = [
     "are you very happy?",
]

In [7]:

outputs = llm.generate(
    prompts,
    sampling_params,
    control_vector_request=ControlVectorRequest("sql_adapter", cv)
)



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.95it/s, est. speed input: 11.74 toks/s, output: 35.21 toks/s]


In [8]:
outputs

[RequestOutput(request_id=0, prompt='are you very happy?', prompt_token_ids=[1, 460, 368, 1215, 4610, 28804], prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\n\ni am very happy to be alive and to be able to help people.', token_ids=[13, 13, 28710, 837, 1215, 4610, 298, 347, 8630, 304, 298, 347, 2358, 298, 1316, 905, 28723, 2], cumulative_logprob=-18.870637419167906, logprobs=None, finish_reason=stop, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1718794603.1927803, last_token_time=1718794603.1927803, first_scheduled_time=1718794603.1997533, first_token_time=1718794603.2321966, time_in_queue=0.0069730281829833984, finished_time=1718794603.7104766), lora_request=None)]